<a href="https://colab.research.google.com/github/neimarcos/Mestrado/blob/main/testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.2 MB/s 


In [ ]:
import pulp 
import networkx as nx
from pprint import pprint


g = nx.Graph()
pos = {0: (0,0), 1:(1,0) , 2: (2,0), 3: (3, 0), 4: (4, 0), 5: (2, -1), 6: (2, -2), 7: (3, -1), 8: (3, 1), 9: (4, 1)}
node_labels = { 0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G", 7: "H", 8: "I", 9: "J"}

g.add_nodes_from(node_labels)
g.add_edges_from([(0, 1),(1, 2),(2, 3),(3, 4),(2, 5),(2, 7),(5, 6),(2, 8),(8, 9)])

Cost= {}
for i in g.nodes:
  Cost[i]=len(g.edges(i));

pprint(Cost)
pprint(g.nodes())

#The problem
prob = pulp.LpProblem("MinimalProbes", pulp.LpMinimize)

# The variables
MP = pulp.LpVariable.dicts("MP", g.nodes(), cat=pulp.LpBinary)
P = pulp.LpVariable.dicts("P", g.edges(), cat=pulp.LpBinary)

prob +=  pulp.lpSum([g.edges[i]*Cost[i] for i in g.nodes]), "Sum_of_Transporting_Costs"


#The objective function

#for i in MP:
#  prob +=  pulp.lpSum(MP * (len(g.edges(i))))



#for (u,v) in g.edges():
prob += pulp.lpSum(MP) 



#The constraints
for (u,v) in g.edges():
    prob += P[(u,v)] <= MP[v]
    prob += P[(u,v)] <= 1-MP[u]
for v in g.nodes():
    prob +=  pulp.lpSum([P[(u,v)] for u in g.neighbors(v) if (u,v) in P]) + pulp.lpSum([P[(v,u)] for u in g.neighbors(v) if (v,u) in P]) >= 2*(1-MP[v]) 
prob.solve()

# The problem data is written to an .lp file
prob.writeLP("ProbesPlacement.lp")

# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    #if v.varValue > 0:
    print(v.name, "=", v.varValue)

{0: 1, 1: 2, 2: 5, 3: 2, 4: 1, 5: 2, 6: 1, 7: 1, 8: 2, 9: 1}
NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9))


TypeError: ignored

In [ ]:
for i in MP:
  print (str(i) + '= ' + str((len(g.edges(i)))))

0= 1
1= 2
2= 5
3= 2
4= 1
5= 2
6= 1
7= 1
8= 2
9= 1
